## 加载工具包

In [1]:
import time
import re
import math
import numpy as np
from six.moves import urllib
import tensorflow as tf

## 定义网络结构

参看VGG网络结构，适当减少了参数尺寸。

In [2]:
#Below code was referred to http://danijar.com/structuring-your-tensorflow-models/
import functools
TOWER_NAME = 'tower'
def define_scope(function):
    attribute = '_cache_' + function.__name__

    @property
    @functools.wraps(function)
    def decorator(self):
        if not hasattr(self, attribute):
           # with tf.variable_scope(function.__name__):
                setattr(self, attribute, function(self))
        return getattr(self, attribute)

    return decorator

In [118]:
#Define dog cat classification model
class imageClassification:
    '''Define a basic model for image classification, the model
    Provides graph structure of tensorflow'''
    
    def __init__(self, image_holder, label_holder, keep_prob, batch_size=32, num_class=1, is_training=True):
        self.image_holder = image_holder
        self.label_holder = label_holder
        self.num_class = num_class
        self.is_training = is_training
        self.weights = []
        self.batch_size = batch_size
        self.keep_prob = keep_prob
        self.prediction
        self.optimizer
        self.accuracy
        self.correct_num
        print('Initializing Image Classification Model!') 
    
    @define_scope
    def sigmoid(self):
        '''Calculate final softmax logits of 
        convolutional neural network'''
        logits = self.build_network
        return tf.nn.sigmoid(logits)
        
    @define_scope
    def prediction(self):
        '''Calculate final softmax logits of 
        convolutional neural network'''
        sigmoid = self.sigmoid
        predictions = sigmoid > 0.5
        return tf.cast(predictions, tf.int32)
    
    @define_scope
    def cost(self):
        #labels = tf.one_hot(self.label_holder, self.num_class, 1, 0)
        labels = tf.cast(self.label_holder, tf.float32)
        #Cross entropy
        #cross_entropy = -tf.reduce_mean(labels*
                                       #tf.log(self.prediction), name='cross_entropy')
        clipped_value = tf.clip_by_value(self.sigmoid, 1e-10, 0.999)
        cross_entropy = -tf.reduce_mean(labels*tf.log(clipped_value) + (1-labels)*tf.log(1-clipped_value))
        
        #Regularization
        l2_loss = cross_entropy
        for i in range(len(self.weights)):
                l2_loss += tf.nn.l2_loss(self.weights[i])
        
        return l2_loss
    
    @define_scope
    def optimizer(self):
        '''Define cross entropy loss function and regularization'''      
        l2_loss = self.cost
        #Decaying Learning Rate
        cur_step = tf.Variable(0, trainable=False)  # count the number of steps taken.
        starter_learning_rate = 0.0001
        learning_rate = tf.train.exponential_decay(starter_learning_rate, cur_step, 2000, 0.96, staircase=True)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        #optimizer = tf.train.AdamOptimizer(1e-4)
        return optimizer.minimize(l2_loss, global_step=cur_step)
    
    @define_scope
    def accuracy(self):
        '''Calculate accuracy for each epoch of training'''
        #labels = tf.one_hot(self.label_holder, self.num_class, 1, 0)
        correct_prediction = tf.equal(self.label_holder, 
                                      self.prediction)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        return accuracy
    
    @define_scope
    def correct_num(self):
        '''Count correct predictions for testing part'''
        #labels = tf.one_hot(self.label_holder, self.num_class, 1, 0)
        correct_prediction = tf.equal(self.label_holder, 
                                      self.prediction)
        correct_ones = tf.reduce_sum(tf.cast(correct_prediction, tf.int32))
        return correct_ones
    
    def _conv_layer(self, input_tensor, shape, wd, scope, is_pooling=True): 
        '''
        Create a layer of convolutional neural network
        Args:
        input_tensor:input, [batch_size, height, width, channel]
        shape: shape of weights, [filter_width, filter_height, filter_channel, output_channel]
        wd: decaying weight
        scope: variable scope
        '''
        #Get the output dim
        output_dim = shape[-1]
        #Create variables
        kernel = self._variable_with_weight_decay('weights',
                                                 shape=shape,
                                                 wd=wd)
        #Compute convolution
        conv = tf.nn.conv2d(input_tensor, kernel, [1, 1, 1, 1], padding='SAME')
        biases = self._variable_on_cpu('biases', [output_dim], tf.constant_initializer(0.0))
        z = tf.nn.bias_add(conv, biases)
        #Batch normalization
        bn = self._batch_normalization(z)
        activation = tf.nn.relu(bn, name=scope.name)
        
        self._activation_summary(activation)
        if is_pooling:
            pool = tf.nn.max_pool(activation, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],
                               padding='SAME', name='pool')
        else:
            pool = activation
        return pool
    
    
    def _fully_layer(self, input_data, out_dim, scope, wd=0.004):
        '''
        Create a fully connected layer with specific parameters
        Args:
        input_data:input tensor, batch_size*width*height*channels
        out_dim: output dimension
        wd: punishment for l2 regularization
        '''
        reshape_tensor = tf.reshape(input_data, [self.batch_size, -1], name='reshape_tensor')
        in_dim = reshape_tensor.get_shape()[1].value
        weights = self._variable_with_weight_decay('weights', shape=[in_dim, out_dim], wd=0.004)
        biases = self._variable_on_cpu('biases', [out_dim], tf.constant_initializer(0.1))
        local = tf.nn.relu(tf.matmul(reshape_tensor, weights) + biases, name=scope.name)
        self._activation_summary(local)
        return local
    
    def _batch_normalization(self, z):
        '''
        Calculate batch nomralization of the input data
        Args:
        z: weighted linear sums of previous neurons
        '''
        decay = 0.997
        epsilon = 0.001
        #Scale mean value
        bias_shape = z.get_shape()[-1]
        gamma = tf.get_variable(name='gamma', initializer=tf.ones(bias_shape))
        beta = tf.get_variable(name='beta', initializer=tf.zeros(bias_shape))
        moving_mean = tf.get_variable(name='moving_mean', initializer=tf.zeros(bias_shape), trainable=False)
        moving_var = tf.get_variable(name='moving_var', initializer=tf.ones(bias_shape), trainable=False)
        axis = list(range(len(z.get_shape()) - 1))
        #If training
        if self.is_training:
            batch_mean, batch_var = tf.nn.moments(z, axis)
            train_mean = tf.assign(moving_mean,
                                   moving_mean * decay + batch_mean * (1 - decay))
            train_var = tf.assign(moving_var,
                                  moving_var * decay + batch_var * (1 - decay))
            with tf.control_dependencies([train_mean, train_var]):
                bn = tf.nn.batch_normalization(z,
                                           batch_mean, batch_var, beta, gamma, epsilon)    
        else:
            bn = tf.nn.batch_normalization(z, moving_mean, moving_var, beta, gamma, epsilon)
        return bn
    
    def _conv_base(self, input_data):
        # conv1
        with tf.variable_scope('conv1') as scope:
            shape = [3, 3, 3, 128]
            wd = 0
            #32*32*32
            pool1 = self._conv_layer(input_data, shape, wd, scope)
        # conv2
        with tf.variable_scope('conv2') as scope:
            wd = 0
            shape = [3, 3, 128, 128]
            #16*16*64
            pool2 = self._conv_layer(pool1, shape, wd, scope)
        # conv3
        with tf.variable_scope('conv3') as scope:
            wd = 0
            shape = [3, 3, 64, 128]
            #8*8*128
            #pool3 = self._conv_layer(pool2, shape, wd, scope)
        activation = pool2
        return activation
    
    def _inception_base(self, input_data):
        with tf.variable_scope('Mixed_5c'):
            with tf.variable_scope('Branch_0') as scope:
                #branch_0 = slim.conv2d(input_data, 64, [1, 1], scope='Conv2d_0a_1x1')
                shape = [1, 1, 128, 64]
                wd = 0
                #8*8*64
                branch_0 = self._conv_layer(input_data, shape, wd, scope, is_pooling=False)
            with tf.variable_scope('Branch_1') as scope:
                with tf.variable_scope('conv_1X1'):
                    shape = [1, 1, 128, 48]
                    #8*8*48
                    branch_1 = self._conv_layer(input_data, shape, wd, scope, is_pooling=False)
                with tf.variable_scope('conv_5X5'):
                    shape = [5, 5, 48, 64]
                    #8*8*64
                    branch_1 = self._conv_layer(branch_1, shape, wd, scope, is_pooling=False)
            with tf.variable_scope('Branch_2') as scope:
                with tf.variable_scope('conv_1X1'):
                    shape = [1, 1, 128, 64]
                    #8*8*64
                    branch_2 = self._conv_layer(input_data, shape, wd, scope, is_pooling=False)
                with tf.variable_scope('conv_3X3_0'):
                    shape = [3, 3, 64, 96]
                    #8*8*96
                    branch_2 = self._conv_layer(branch_2, shape, wd, scope, is_pooling=False)
                with tf.variable_scope('conv_3X3_1'):
                    shape= [3, 3, 96, 96]
                    #8*8*96
                    branch_2 = self._conv_layer(branch_2, shape, wd, scope, is_pooling=False)
            with tf.variable_scope('Branch_3') as scope:
                #branch_3 = slim.avg_pool2d(input_data, [3, 3], scope='AvgPool_0a_3x3')
                #branch_3 = slim.conv2d(branch_3, 64, [1, 1], scope='Conv2d_0b_1x1')
                #8*8*128
                branch_3 = tf.nn.avg_pool(input_data, ksize=[1, 3, 3, 1], strides=[1, 1, 1, 1],
                                                                   padding='SAME', name='pool')
                shape = [1, 1, 128, 64]
                #8*8*64
                branch_3 = self._conv_layer(branch_3, shape, wd, scope, is_pooling=False)
            net = tf.concat([branch_0, branch_1, branch_2, branch_3], 3) 
        return net
    
    @define_scope
    def build_network(self):
        """Build the clasification network.
        Args:
        images: Images returned from distorted_inputs() or inputs().
        Returns:
        Logits.
        """
        # We instantiate all variables using tf.get_variable() instead of
        # tf.Variable() in order to share variables across multiple GPU training runs.
        # If we only ran this model on a single GPU, we could simplify this function
        # by replacing all instances of tf.get_variable() with tf.Variable().
        #
        output = self._conv_base(self.image_holder)
        net = self._inception_base(output)
        #Flatten the output
        with tf.variable_scope('AveragePooling'):
            net = tf.nn.avg_pool(net, ksize=[1, 16, 16, 1], strides=[1, 1, 1, 1],
                                 padding='VALID', name='pool')
        #1X1X288
        net = tf.nn.dropout(net, self.keep_prob)
        

        # local3
        #with tf.variable_scope('fully_connected1') as scope:
            #local3 = self._fully_layer(net, 384, scope)
        #local3_dropout = tf.nn.dropout(local3, self.keep_prob)
        # local4
        #with tf.variable_scope('fully_connected2') as scope:
            #local4 = self._fully_layer(pool3_dropout, 192, scope)
            
        #local4_dropout = tf.nn.dropout(local4, self.keep_prob)
        
        # linear layer(WX + b),
        # We don't apply softmax here because
        # tf.nn.sparse_softmax_cross_entropy_with_logits accepts the unscaled logits
        # and performs the softmax internally for efficiency.
        #output = local3_dropout
        with tf.variable_scope('sigmoid_linear') as scope:
            weights = self._variable_with_weight_decay('weights', [1, 1, 288, self.num_class],
                                                  wd=0.0)
            biases = self._variable_on_cpu('biases', [self.num_class],
                                      tf.constant_initializer(0.0))
            conv = tf.nn.conv2d(net, weights, [1, 1, 1, 1], padding='SAME')
            sigmoid_linear =  tf.nn.bias_add(conv, biases, name=scope.name)
            #remove dimension which is 1
            sigmoid_linear = tf.squeeze(sigmoid_linear,[1, 2], name='SpatialSqueeze')
            self._activation_summary(sigmoid_linear)
            #print(sigmoid_linear)
        return sigmoid_linear
    
    #def _conv_layer(self, )
    
    def _activation_summary(self, x):
        """Helper to create summaries for activations.
        Creates a summary that provides a histogram of activations.
        Creates a summary that measures the sparsity of activations.
        Args:
        x: Tensor
        Returns:
        nothing
        """
        # Remove 'tower_[0-9]/' from the name in case this is a multi-GPU training
        # session. This helps the clarity of presentation on tensorboard.
        tensor_name = re.sub('%s_[0-9]*/' % TOWER_NAME, '', x.op.name)
        tf.summary.histogram(tensor_name + '/activations', x)
        tf.summary.scalar(tensor_name + '/sparsity',
                          tf.nn.zero_fraction(x))
    
    def _variable_on_cpu(self, name, shape, initializer):
        """Helper to create a Variable stored on CPU memory.
        Args:
        name: name of the variable
        shape: list of ints
        initializer: initializer for Variable
        Returns:
        Variable Tensor
        """
        with tf.device('/cpu:0'):
            with tf.variable_scope('weights'):
                dtype = tf.float32
                var = tf.get_variable(initializer=initializer(shape), dtype=dtype, name=name)
            #var = tf.get_variable(name, shape, initializer=initializer, dtype=dtype)
        return var

    def _variable_with_weight_decay(self, name, shape, wd):
        """Helper to create an initialized Variable with weight decay.
        Note that the Variable is initialized with a truncated normal distribution.
        A weight decay is added only if one is specified.
        Args:
        name: name of the variable
        shape: list of ints
        stddev: standard deviation of a truncated Gaussian
        wd: add L2Loss weight decay multiplied by this float. If None, weight
        decay is not added for this Variable.
        Returns:
        Variable Tensor
        """
        dtype = tf.float32
        var = self._variable_on_cpu(name, shape,
                               tf.contrib.layers.xavier_initializer())
        if wd is not None and wd > 0:
            weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, name='weight_loss')
            #tf.add_to_collection('losses', weight_decay)
            self.weights.append(weight_decay)
        return var

## 定义输入张量

In [119]:
batch_size = 50
tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():
    image_holder = tf.placeholder(tf.float32, [batch_size, 64, 64, 3])
    label_holder = tf.placeholder(tf.int32, [batch_size, 1])
    keep_prob = tf.placeholder(tf.float32)
    model = imageClassification(image_holder, label_holder, keep_prob, batch_size, num_class =1)
    tf.get_variable_scope().reuse_variables()
    test_model = imageClassification(image_holder, label_holder, keep_prob, batch_size, 1, False)
    #with tf.variable_scope('VGG'):
        #model_train = imageClassification(image_holder, label_holder, keep_prob, batch_size=64, num_class =1)
    #with tf.variable_scope('VGG', reuse=True):
        #model_test = imageClassification(image_holder, label_holder, keep_prob, batch_size=16, num_class =1) 
    saver = tf.train.Saver()

Initializing Image Classification Model!
Initializing Image Classification Model!


## 处理训练数据和测试数据

In [107]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)
path = 'C:/Users/onlooker/Documents/deeplearning_projects/cats_dogs_classification/dataset'
training_set = train_datagen.flow_from_directory(path+'/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = batch_size,
                                                 class_mode = 'binary')


Found 8000 images belonging to 2 classes.


In [108]:
test_set = test_datagen.flow_from_directory(path+'/test_set', shuffle=False,
                                            target_size = (64, 64),
                                            batch_size = batch_size,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## 训练和测试

In [120]:
import os
num_steps = 2001
epochs = 10
with tf.Session(graph=graph) as sess:
    init_op = tf.global_variables_initializer()
    #If there exists models in advance, load them
    if os.path.exists("./Model/checkpoint"): # 注意此处路径前添加"./" ')
        saver.restore(sess, "./Model/model.ckpt") # 注意此处路径前添加"./" 
    else:
        sess.run(init_op)
    for _ in range(epochs):
        for step in range(num_steps):
            #产生训练用样本集      
            batch_data, batch_labels = training_set.next()
            batch_labels = np.reshape(batch_labels, [-1,1])
            #数据传递给tensorflow
            feed_dict = {image_holder: batch_data, label_holder : batch_labels, keep_prob: 0.5}
            _, loss = sess.run([model.optimizer, model.cost], feed_dict=feed_dict)
            if step%200 == 0:
                #每50次计算准确度
                #result = sess.run(cross_entropy, feed_dict={x : batch_data, y_ : batch_labels, keep_prob:1})
                print('Cost:', loss) 
                #保存模型
        saver.save(sess, "Model/model.ckpt")


INFO:tensorflow:Restoring parameters from ./Model/model.ckpt


INFO:tensorflow:Restoring parameters from ./Model/model.ckpt


Cost: 0.431856
Cost: 0.366955
Cost: 0.400778
Cost: 0.383108
Cost: 0.328595
Cost: 0.382513
Cost: 0.383736
Cost: 0.387899
Cost: 0.371837
Cost: 0.3643
Cost: 0.324861
Cost: 0.246209
Cost: 0.257335
Cost: 0.341806
Cost: 0.265123
Cost: 0.46841
Cost: 0.314234
Cost: 0.32488
Cost: 0.46152
Cost: 0.38794
Cost: 0.323661
Cost: 0.279217
Cost: 0.307353
Cost: 0.378528
Cost: 0.454374
Cost: 0.336229
Cost: 0.425698
Cost: 0.32699
Cost: 0.383612
Cost: 0.38041
Cost: 0.370629
Cost: 0.365638
Cost: 0.352378
Cost: 0.252727
Cost: 0.483972
Cost: 0.402577
Cost: 0.346532
Cost: 0.329065
Cost: 0.334837
Cost: 0.334728


KeyboardInterrupt: 

In [121]:
with tf.Session(graph=graph) as sess:
    saver.restore(sess, "./Model/model.ckpt") # 注意此处路径前添加"./" 
    print('Testing Result.....')  
    count = 0
    for i in range(40):
        batch_data, batch_labels = test_set.next()
        #print(batch_data.shape)
        #print(i)
        batch_labels = np.reshape(batch_labels, [-1,1])
        feed_dict = {image_holder: batch_data, label_holder : batch_labels, keep_prob: 1}
        c, a = sess.run([test_model.correct_num, test_model.prediction], feed_dict=feed_dict)
        #print(a)
        count += c
    result = count*1.0/2000
    print('Accuracy:', result)

INFO:tensorflow:Restoring parameters from ./Model/model.ckpt


INFO:tensorflow:Restoring parameters from ./Model/model.ckpt


Testing Result.....
Accuracy: 0.8495
